In [ ]:
pip install tensorflow opencv-python numpy qrcode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
import qrcode


In [ ]:
def generate_qr_code_data(num_samples):
    X = []
    Y = []

    for _ in range(num_samples):
        qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_L)
        data = "Your QR code data here"  # Replace with your QR code data
        qr.add_data(data)
        qr.make(fit=True)
        img = qr.make_image(fill_color="black", back_color="white").convert("RGB")
        img = np.array(img)
        X.append(img)
        Y.append(data)

    X = np.array(X) / 255.0
    return X, Y

num_samples = 1000  # Adjust as needed
X_train, Y_train = generate_qr_code_data(num_samples)


In [ ]:
input_shape = X_train[0].shape

def create_qr_code_recognition_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)  # For binary classification

    model = Model(inputs, outputs)
    return model

model = create_qr_code_recognition_model(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the QR code data as numerical labels
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)

# Make sure to use Y_train_encoded for training
model.fit(X_train, Y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
25/25 [==============================] - 156s 6s/step - loss: 0.0245 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
25/25 [==============================] - 172s 7s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
25/25 [==============================] - 154s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
25/25 [==============================] - 151s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
25/25 [==============================] - 154s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
25/25 [==============================] - 161s 7s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
25/25 [==============================] - 152s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_l

In [ ]:
def recognize_qr_code(image_path):
    # Check if the image file exists
    if not os.path.isfile(image_path):
        return "Image file does not exist."

    img = cv2.imread(image_path)

    # Check if img is None (indicating failure to read the image)
    if img is None:
        return "Failed to read the image."

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, input_shape[:2])
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)

    if prediction > 0.5:
        return "QR code detected: " + Y_train[np.argmax(model.predict(X_train))]
    else:
        return "No QR code detected."

image_path = "path/to/your/image.jpg"  # Replace with the path to your test image
result = recognize_qr_code(image_path)
print(result)


NameError: ignored